In [1]:
import hashlib
import numpy as np
import aeskeyschedule
from tqdm import tqdm

# DFA by Piret and Quisquater

This template is constructed to aid with implementing both the `simple` and `full` DFA variants needed for the homework.

In [2]:
SBOX = np.array([
        0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76,
        0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0,
        0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15,
        0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75,
        0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84,
        0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF,
        0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8,
        0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2,
        0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73,
        0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB,
        0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79,
        0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08,
        0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A,
        0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E,
        0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,
        0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16
], dtype=np.uint8)

# Inverse of AES SBOX
ISBOX = SBOX.argsort()

# AES MixCols matrix
MIXCOLS = np.array([[2, 3, 1, 1],
                    [1, 2, 3, 1],
                    [1, 1, 2, 3],
                    [3, 1, 1, 2]])



In [3]:
# Check if you got the right answer for the 4 keybytes in the first column (key at indices [0, 13, 10, 7])
# Note: We're referring to the round 10 keybytes (no need to rewind with the the key schedule)
# Note: The same key is used both for the `simple` and `full` DFA

def check_keybytes(k_0: int, k_13: int, k_10: int, k_7: int):
    keybytes = bytes([k_0, k_13, k_10, k_7])
    hasher = hashlib.sha3_256()
    hasher.update(keybytes)
    key_hash = hasher.hexdigest()
    if key_hash == '4409976e63e88e6d0ef93405e6b6d678c2a498d22dcaa72b28c8c9cd6233ec7f':
        print("Congratulations! Correct 4 keybytes found")
        return True
    
    print("Not quite right")
    return False

In [4]:
# Two pairs of ciphertext/faulty texts.
# The fault is injected in the *first byte* before the MixCols in the 9th round
# Note: use for `simple` DFA (part C)

simple_ctxt1 = [174, 44, 204, 43, 18, 196, 238, 88, 3, 227, 92, 0, 137, 106, 205, 88]
simple_ftxt1 = [128, 44, 204, 43, 18, 196, 238, 171, 3, 227, 159, 0, 137, 186, 205, 88]

simple_ctxt2 = [41, 4, 148, 29, 23, 74, 41, 127, 125, 148, 36, 219, 29, 127, 4, 58]
simple_ftxt2 = [186, 4, 148, 29, 23, 74, 41, 160, 125, 148, 59, 219, 29, 172, 4, 58]


# Load ctext/ftext pairs in the correct AES column order
simple_ctxt1 = np.reshape(simple_ctxt1, (4, 4), order='F').astype(np.uint8)
simple_ftxt1 = np.reshape(simple_ftxt1, (4, 4), order='F').astype(np.uint8)

simple_ctxt2 = np.reshape(simple_ctxt2, (4, 4), order='F').astype(np.uint8)
simple_ftxt2 = np.reshape(simple_ftxt2, (4, 4), order='F').astype(np.uint8)

print("First pair diff:")
print(simple_ctxt1 ^ simple_ftxt1)

print("Second pair diff:")
print(simple_ctxt2 ^ simple_ftxt2)
# print(simple_ctxt1)

First pair diff:
[[ 46   0   0   0]
 [  0   0   0 208]
 [  0   0 195   0]
 [  0 243   0   0]]
Second pair diff:
[[147   0   0   0]
 [  0   0   0 211]
 [  0   0  31   0]
 [  0 223   0   0]]


In [5]:
# Load all plaintexts/ciphertexts/faultytexts in the correct AES column order
# Note: not needed for the `simple` DFA (part C) 

row_type = np.dtype((np.uint8, (4, 4)))
all_ctext = np.fromfile("full_dfa_data/ctext.bin", dtype=row_type).transpose(0, 2, 1)
all_ptext = np.fromfile("full_dfa_data/ptext.bin", dtype=row_type).transpose(0, 2, 1)
all_ftext = np.fromfile("full_dfa_data/ftext.bin", dtype=row_type).transpose(0, 2, 1)

print(all_ctext[0] ^ all_ftext[0])

[[  0   0   0 109]
 [  0   0  50   0]
 [  0   3   0   0]
 [206   0   0   0]]


In [6]:
# Galois multiplication by 2 (for MixCols)
def galois_mult_2(a):
    temp = (a << 1) & 0xff

    if (a & 0x80):
        temp ^= 0x1b

    return temp

# Galois multiplication by 3 (for MixCols)
def galois_mult_3(a):
    return galois_mult_2(a) ^ a

In [7]:
# AES ShiftRows
# `mat` is 4x4 AES matrix
def shift(mat):
    shifted = np.zeros_like(mat)
    for i in range(4):
        shifted[i] = mat[i, np.arange(i, 4+i) % 4]
    return shifted

# TODO: implement your own. Should undo shift(mat)
def unshift(mat):
    # for i in range(3):
    #    mat = shift(mat)
    # return mat
    return shift(shift(shift(mat)))
    # pass


In [8]:
# Precompute all possible mixcols(glitch)
# TODO: Only works if glitch is in the first row! 
#       For `full` DFA need to add more entries to make it work for glitches in any row

D = []

mixcol = MIXCOLS[:, 0]
for x in range(1, 255+1):
    D_element = []
    for j in range(4):
        out = None
        if mixcol[j] == 1:
            out = x
        if mixcol[j] == 2:
            out = galois_mult_2(x)
        if mixcol[j] == 3:
            out = galois_mult_3(x)
        D_element.append(out)
    D.append(D_element)


print("Length of lookup table:", len(D))

Length of lookup table: 255


In [20]:
#full attack D
D_full = []

for i in range(len(MIXCOLS)):
    mixcol = MIXCOLS[:, i]
    for x in range(1, 255+1):
        D_element = []
        for j in range(4):
            out = None
            if mixcol[j] == 1:
                out = x
            if mixcol[j] == 2:
                out = galois_mult_2(x)
            if mixcol[j] == 3:
                out = galois_mult_3(x)
            D_element.append(out)
        D_full.append(D_element)

print("Length of lookup table:", len(D_full))
# print(D_full)

Length of lookup table: 1020


# Simple Attack Algorithm (finds 4 bytes):

## Variable Definitions
Let $c, c'$ be the first (ciphertext, faultytext) pair and $c, c^{*\prime}$ be the second pair.
Note: $c_0$ refers to the $0$ index byte of the ciphertext

## Preliminary Filtering
1. For each $K_0 \in 0 \text{ to } 255$ and $K_{13} \in 0 \text{ to } 255$, and $x \in 1 \text{ to } 255$
    - Check if $ISBOX(K_0 \oplus c_0) \oplus ISBOX(K_0 \oplus c_0') \stackrel{?}{=} D[x, 0]$
    - Check if $ISBOX(K_{13} \oplus c_{13}) \oplus ISBOX(K_{13} \oplus c_{13}') \stackrel{?}{=} D[x, 1]$

    - If both are true, add candidates $\{K_0, K_{13}\}$ to group $K_{c2}$
2. For each $K_{10} \in 0 \text{ to } 255$, and $\{K_{0}, K_{13}\} \in K_{c2}$, and $x \in 1 \text{ to } 255$
    - Check if $ISBOX(K_0 \oplus c_0) \oplus ISBOX(K_0 \oplus c_0') \stackrel{?}{=} D[x, 0]$
    - Check if $ISBOX(K_{13} \oplus c_{13}) \oplus ISBOX(K_{13} \oplus c_{13}') \stackrel{?}{=} D[x, 1]$
    - Check if $ISBOX(K_{10} \oplus c_{10}) \oplus ISBOX(K_{10} \oplus c_{10}') \stackrel{?}{=} D[x, 2]$

    - If so, add candidates $\{K_0, K_{13}, K_{10}\}$ to group $K_{c3}$
3. For each $K_{7} \in 0 \text{ to } 255$, and $\{K_{0}, K_{13}, K_{10}\} \in K_{c3}$, and $x \in 1 \text{ to } 255$
    - Check if $ISBOX(K_0 \oplus c_0) \oplus ISBOX(K_0 \oplus c_0') \stackrel{?}{=} D[x, 0]$
    - Check if $ISBOX(K_{13} \oplus c_{13}) \oplus ISBOX(K_{13} \oplus c_{13}') \stackrel{?}{=} D[x, 1]$
    - Check if $ISBOX(K_{10} \oplus c_{10}) \oplus ISBOX(K_{10} \oplus c_{10}') \stackrel{?}{=} D[x, 2]$
    - Check if $ISBOX(K_{7} \oplus c_{7}) \oplus ISBOX(K_{7} \oplus c_{7}') \stackrel{?}{=} D[x, 3]$

    - If so, add candidates $\{K_0, K_{13}, K_{10}, K_{7}\}$ to group $K_{c4}$
    
## Finding Final Candidate
1. For each $\{K_{0}, K_{13}, K_{10}, K_7\} \in K_{c4}$, and $x \in 1 \text{ to } 255$
  - $ISBOX(K_0 \oplus c^{*}_0) \oplus ISBOX(K_0 \oplus c^{*\prime}_0) \stackrel{?}{=} D[x, 0]$
  - $ISBOX(K_{13} \oplus c^{*}_{13}) \oplus ISBOX(K_{13} \oplus c^{*\prime}_{13}) \stackrel{?}{=} D[x, 1]$
  - $ISBOX(K_{10} \oplus c^{*}_{10}) \oplus ISBOX(K_{10} \oplus c^{*\prime}_{10}) \stackrel{?}{=} D[x, 2]$
  - $ISBOX(K_{7} \oplus c^{*}_{7}) \oplus ISBOX(K_{7} \oplus c^{*\prime}_{7}) \stackrel{?}{=} D[x, 3]$
  - If all are true $\{K_0, K_{13}, K_{10}, K_{7}\}$, are likely correct keybytes


# Indexing Tips:
You can access elements such as $c_{0}, c_{13}, c_{10}, c_{7}$ by using np.unravel() like this:

In [10]:
#preperation for simple attack

c0 = simple_ctxt1[np.unravel_index(0, shape=(4,4), order='F')]
c13 = simple_ctxt1[np.unravel_index(13, shape=(4,4), order='F')]
c10 = simple_ctxt1[np.unravel_index(10, shape=(4,4), order='F')]
c7 = simple_ctxt1[np.unravel_index(7, shape=(4,4), order='F')]

c0_ = simple_ftxt1[np.unravel_index(0, shape=(4,4), order='F')]
c13_ = simple_ftxt1[np.unravel_index(13, shape=(4,4), order='F')]
c10_ = simple_ftxt1[np.unravel_index(10, shape=(4,4), order='F')]
c7_ = simple_ftxt1[np.unravel_index(7, shape=(4,4), order='F')]

c01 = simple_ctxt2[np.unravel_index(0, shape=(4,4), order='F')]
c131 = simple_ctxt2[np.unravel_index(13, shape=(4,4), order='F')]
c101 = simple_ctxt2[np.unravel_index(10, shape=(4,4), order='F')]
c71 = simple_ctxt2[np.unravel_index(7, shape=(4,4), order='F')]

c0_1 = simple_ftxt2[np.unravel_index(0, shape=(4,4), order='F')]
c13_1 = simple_ftxt2[np.unravel_index(13, shape=(4,4), order='F')]
c10_1 = simple_ftxt2[np.unravel_index(10, shape=(4,4), order='F')]
c7_1 = simple_ftxt2[np.unravel_index(7, shape=(4,4), order='F')]


In [11]:
# Perform `simple` DFA below: you should use (`simple_ctxt1`, `simple_ftxt1`) and (`simple_ctxt2`, `simple_ftxt2`)
# Note: remember to use `check_keybytes()` function to check your answer!
def simple_attack():
    #filtering for first 2 bytes
    kc2 = []
    for k0 in tqdm(range(1,256)):
        diff_0 = ISBOX[(k0 ^ c0)]^ISBOX[(k0 ^ c0_)]
        for k13 in range(1,256):
            diff_1 = ISBOX[(k13 ^ c13)]^ISBOX[(k13 ^ c13_)]
            for x in range(len(D)):
                if( D[x][0] == diff_0 and D[x][1] == diff_1):
                    kc2.append((k0, k13))# got 2 possible key values, add candidate
                    break

    #filtering for 3rd bytes
    kc3 = []
    for k0, k13 in tqdm(kc2):
        diff_0 = ISBOX[(k0 ^ c0)]^ISBOX[(k0 ^ c0_)]
        diff_1 = ISBOX[(k13 ^ c13)]^ISBOX[(k13 ^ c13_)]
        for k10 in range(1,256):
            diff_2 = ISBOX[(k10 ^ c10)] ^ ISBOX[(k10 ^ c10_)]
            for x in range(len(D)):
                if( D[x][0] == diff_0 and D[x][1] == diff_1 and D[x][2] == diff_2):
                    kc3.append((k0, k13, k10))# got 3rd possible value, add candidate
                    break
    
    #filtering for 4th bytes
    kc4 = []
    for k0, k13, k10 in tqdm(kc3):
        diff_0 = ISBOX[(k0 ^ c0)]^ISBOX[(k0 ^ c0_)]
        diff_1 = ISBOX[(k13 ^ c13)]^ISBOX[(k13 ^ c13_)]
        diff_2 = ISBOX[(k10 ^ c10)]^ISBOX[(k10 ^ c10_)]
        for k7 in range(1,256):
            diff_3 = ISBOX[(k7 ^ c7)] ^ ISBOX[(k7 ^ c7_)]
            for x in range(len(D)):
                if( D[x][0] == diff_0 and D[x][1] == diff_1 and D[x][2] == diff_2 and D[x][3] == diff_3):
                    kc4.append((k0, k13, k10, k7))# got 4th possible value, add candidate
                    break
    return kc4
kc4 = simple_attack() 
#check to see if all bytes are correct 
for k0,k13,k10,k7 in tqdm(kc4):
    diff_0 = ISBOX[(k0 ^ c01)]^ISBOX[(k0 ^ c0_1)]
    diff_1 = ISBOX[(k13 ^ c131)]^ISBOX[(k13 ^ c13_1)]
    diff_2 = ISBOX[(k10 ^ c101)]^ISBOX[(k10 ^ c10_1)]
    diff_3 = ISBOX[(k7 ^ c71)] ^ ISBOX[(k7 ^ c7_1)]
    for x in range(len(D)):
        if (D[x][0] == diff_0 and D[x][1] == diff_1 and D[x][2] == diff_2 and D[x][3] == diff_3):
            if(check_keybytes(k0,k13,k10,k7)):
                print(k0,k13,k10,k7)
                break      

100%|██████████| 240/240 [00:00<00:00, 10216.51it/s]

Congratulations! Correct 4 keybytes found
168 138 164 45


In [21]:
#to segregate columns into groups
get_groups = [[], [], [], []]
for i in range(len(all_ctext)):
    group = all_ctext[i] ^ all_ftext[i]
    for j in range(4):
        if group[0][j]:
            get_groups[j].append(i)
            
get_groups[0] = str(get_groups[0])#falut in first column
get_groups[1] = str(get_groups[1])#falut in second column
get_groups[2] = str(get_groups[2])#falut in third column
get_groups[3] = str(get_groups[3])#falut in fourth column
print("\n".join(get_groups))

[18, 22, 35, 38, 48, 69, 77, 96, 118, 126, 139, 141, 151, 159, 169, 180, 211, 212, 217, 225, 228, 257, 260, 295, 303, 326, 335, 343, 344, 352, 362, 374, 376, 379, 381, 385, 395, 408, 412, 419, 422, 430, 434, 440, 448, 461, 467, 472, 488, 493, 504, 510, 513, 516, 517, 524, 528, 541, 546, 548, 572, 582, 607, 620, 626, 630, 631, 633, 635, 654, 663, 692, 694, 705, 717, 728, 737, 750, 753, 754, 762, 768, 782, 789, 797, 804, 815, 822, 833, 834, 837, 843, 855, 869, 889, 894, 912, 926, 933, 942, 972, 984, 986]
[4, 5, 9, 23, 30, 33, 37, 52, 54, 64, 91, 94, 95, 98, 101, 102, 113, 127, 153, 155, 157, 161, 163, 165, 171, 178, 218, 219, 230, 236, 243, 250, 252, 255, 277, 292, 317, 319, 321, 339, 375, 380, 406, 417, 433, 442, 447, 449, 453, 477, 487, 497, 561, 571, 576, 581, 590, 592, 593, 594, 595, 609, 622, 629, 639, 656, 657, 659, 666, 673, 683, 699, 724, 735, 763, 769, 796, 799, 802, 806, 819, 844, 863, 872, 884, 890, 895, 898, 910, 919, 948, 957, 974, 982, 995]
[24, 28, 56, 62, 71, 73, 75, 103,

In [30]:
# Perform `full` DFA below: you should use `all_ctext` and `all_ftext`
# Note: You can still use `check_keybytes()` function to check your answer for the first 4 keybytes
# Note: After recovering the whole key, you need to rewind the key schedule to get the original key
#       You can you use the python library `aeskeyschedule` (pip install aeskeyschedule) if you wish

def full_attack():

    #filtering for first 2 bytes
    kc2 = []
    for k0 in tqdm(range(1,256)):
        diff_0 = ISBOX[(k0 ^ c0)]^ISBOX[(k0 ^ c0_)] 
        for k13 in range(1,256):
            diff_1 = ISBOX[(k13 ^ c13)]^ISBOX[(k13 ^ c13_)]
            for x in range(len(D_full)):
                if( D_full[x][0] == diff_0 and D_full[x][1] == diff_1):
                    kc2.append((k0, k13))# got 2 possible key values, add candidate
                    break
    
    #filtering for 3rd bytes
    kc3 = []
    for k0, k13 in tqdm(kc2):
        diff_0 = ISBOX[(k0 ^ c0)]^ISBOX[(k0 ^ c0_)]
        diff_1 = ISBOX[(k13 ^ c13)]^ISBOX[(k13 ^ c13_)]
        for k10 in range(1,256):
            diff_2 = ISBOX[(k10 ^ c10)] ^ ISBOX[(k10 ^ c10_)]
            for x in range(len(D_full)):
                if( D_full[x][0] == diff_0 and D_full[x][1] == diff_1 and D_full[x][2] == diff_2):
                    kc3.append((k0, k13, k10)) # got 3rd possible value, add candidate
                    break
    
    #filtering for 4th bytes
    kc4 = []
    for k0, k13, k10 in tqdm(kc3):
        diff_0 = ISBOX[(k0 ^ c0)]^ISBOX[(k0 ^ c0_)]
        diff_1 = ISBOX[(k13 ^ c13)]^ISBOX[(k13 ^ c13_)]
        diff_2 = ISBOX[(k10 ^ c10)]^ISBOX[(k10 ^ c10_)]
        for k7 in range(1,256):
            diff_3 = ISBOX[(k7 ^ c7)] ^ ISBOX[(k7 ^ c7_)]
            for x in range(len(D_full)):
                if( D_full[x][0] == diff_0 and D_full[x][1] == diff_1 and D_full[x][2] == diff_2 and D_full[x][3] == diff_3):
                    kc4.append((k0, k13, k10, k7))# got 4th possible value, add candidate
                    break

    print("checking for correct keys")
    for k0,k13,k10,k7 in tqdm(kc4):
        diff_0 = ISBOX[(k0 ^ c01)]^ISBOX[(k0 ^ c0_1)]
        diff_1 = ISBOX[(k13 ^ c131)]^ISBOX[(k13 ^ c13_1)]
        diff_2 = ISBOX[(k10 ^ c101)]^ISBOX[(k10 ^ c10_1)]
        diff_3 = ISBOX[(k7 ^ c71)] ^ ISBOX[(k7 ^ c7_1)]
        #check to see if all bytes are correct
        for x in range(len(D_full)):
            if (D_full[x][0] == diff_0 and D_full[x][1] == diff_1 and D_full[x][2] == diff_2 and D_full[x][3] == diff_3):
                # if(check_keybytes(k0,k13,k10,k7)):
                print(k0,k13,k10,k7)
                #     print(diff_0,diff_1,diff_2,diff_3)
                # else: 
                #     print("not found")
                break

In [31]:
#preperation for full attack

#for first column
simple_ctxt1 = all_ctext[22]
simple_ftxt1 = all_ftext[22]
simple_ctxt2 = all_ctext[18]
simple_ftxt2 = all_ftext[18]

c0 = simple_ctxt1[np.unravel_index(0, shape=(4,4), order='F')]
c13 = simple_ctxt1[np.unravel_index(13, shape=(4,4), order='F')]
c10 = simple_ctxt1[np.unravel_index(10, shape=(4,4), order='F')]
c7 = simple_ctxt1[np.unravel_index(7, shape=(4,4), order='F')]

c0_ = simple_ftxt1[np.unravel_index(0, shape=(4,4), order='F')]
c13_ = simple_ftxt1[np.unravel_index(13, shape=(4,4), order='F')]
c10_ = simple_ftxt1[np.unravel_index(10, shape=(4,4), order='F')]
c7_ = simple_ftxt1[np.unravel_index(7, shape=(4,4), order='F')]

c01 = simple_ctxt2[np.unravel_index(0, shape=(4,4), order='F')]
c131 = simple_ctxt2[np.unravel_index(13, shape=(4,4), order='F')]
c101 = simple_ctxt2[np.unravel_index(10, shape=(4,4), order='F')]
c71 = simple_ctxt2[np.unravel_index(7, shape=(4,4), order='F')]

c0_1 = simple_ftxt2[np.unravel_index(0, shape=(4,4), order='F')]
c13_1 = simple_ftxt2[np.unravel_index(13, shape=(4,4), order='F')]
c10_1 = simple_ftxt2[np.unravel_index(10, shape=(4,4), order='F')]
c7_1 = simple_ftxt2[np.unravel_index(7, shape=(4,4), order='F')]

print("first 4 bytes")
full_attack()


first 4 bytes


100%|██████████| 992/992 [00:59<00:00, 16.61it/s]


checking for correct keys


 37%|███▋      | 354/952 [00:00<00:00, 3525.54it/s]

168 138 164 45


100%|██████████| 952/952 [00:00<00:00, 3657.43it/s]


In [29]:
#preperation for full attack

#for first column
simple_ctxt1 = all_ctext[22]
simple_ftxt1 = all_ftext[22]
simple_ctxt2 = all_ctext[18]
simple_ftxt2 = all_ftext[18]

c0 = simple_ctxt1[np.unravel_index(0, shape=(4,4), order='F')]
c13 = simple_ctxt1[np.unravel_index(13, shape=(4,4), order='F')]
c10 = simple_ctxt1[np.unravel_index(10, shape=(4,4), order='F')]
c7 = simple_ctxt1[np.unravel_index(7, shape=(4,4), order='F')]

c0_ = simple_ftxt1[np.unravel_index(0, shape=(4,4), order='F')]
c13_ = simple_ftxt1[np.unravel_index(13, shape=(4,4), order='F')]
c10_ = simple_ftxt1[np.unravel_index(10, shape=(4,4), order='F')]
c7_ = simple_ftxt1[np.unravel_index(7, shape=(4,4), order='F')]

c01 = simple_ctxt2[np.unravel_index(0, shape=(4,4), order='F')]
c131 = simple_ctxt2[np.unravel_index(13, shape=(4,4), order='F')]
c101 = simple_ctxt2[np.unravel_index(10, shape=(4,4), order='F')]
c71 = simple_ctxt2[np.unravel_index(7, shape=(4,4), order='F')]

c0_1 = simple_ftxt2[np.unravel_index(0, shape=(4,4), order='F')]
c13_1 = simple_ftxt2[np.unravel_index(13, shape=(4,4), order='F')]
c10_1 = simple_ftxt2[np.unravel_index(10, shape=(4,4), order='F')]
c7_1 = simple_ftxt2[np.unravel_index(7, shape=(4,4), order='F')]

print("first 4 bytes")
full_attack()


first 4 bytes


100%|██████████| 992/992 [00:59<00:00, 16.63it/s]


checking for correct keys


100%|██████████| 952/952 [00:00<00:00, 3229.96it/s]

168 138 164 45


In [26]:
#preperation for full attack

#for second column
simple_ctxt1 = all_ctext[4]
simple_ftxt1 = all_ftext[4]
simple_ctxt2 = all_ctext[819]
simple_ftxt2 = all_ftext[819]
c0 = simple_ctxt1[np.unravel_index(4, shape=(4,4), order='F')]
c13 = simple_ctxt1[np.unravel_index(1, shape=(4,4), order='F')]
c10 = simple_ctxt1[np.unravel_index(14, shape=(4,4), order='F')]
c7 = simple_ctxt1[np.unravel_index(11, shape=(4,4), order='F')]

c0_ = simple_ftxt1[np.unravel_index(4, shape=(4,4), order='F')]
c13_ = simple_ftxt1[np.unravel_index(1, shape=(4,4), order='F')]
c10_ = simple_ftxt1[np.unravel_index(14, shape=(4,4), order='F')]
c7_ = simple_ftxt1[np.unravel_index(11, shape=(4,4), order='F')]

c01 = simple_ctxt2[np.unravel_index(4, shape=(4,4), order='F')]
c131 = simple_ctxt2[np.unravel_index(1, shape=(4,4), order='F')]
c101 = simple_ctxt2[np.unravel_index(14, shape=(4,4), order='F')]
c71 = simple_ctxt2[np.unravel_index(11, shape=(4,4), order='F')]

c0_1 = simple_ftxt2[np.unravel_index(4, shape=(4,4), order='F')]
c13_1 = simple_ftxt2[np.unravel_index(1, shape=(4,4), order='F')]
c10_1 = simple_ftxt2[np.unravel_index(14, shape=(4,4), order='F')]
c7_1 = simple_ftxt2[np.unravel_index(11, shape=(4,4), order='F')]
print("second four bytes")
full_attack()

second four bytes


100%|██████████| 996/996 [01:02<00:00, 15.99it/s]


checking for correct keys


 81%|████████  | 763/944 [00:00<00:00, 3799.15it/s]

53 73 46 171


100%|██████████| 944/944 [00:00<00:00, 3623.28it/s]


In [27]:
#preperation for full attack

#for third column
simple_ctxt1 = all_ctext[24]
simple_ftxt1 = all_ftext[24]
simple_ctxt2 = all_ctext[28]
simple_ftxt2 = all_ftext[28]

# print(simple_ctxt1)
# print(simple_ftxt1)
# print(simple_ctxt1^simple_ftxt1)
# print(simple_ctxt2)
# print(simple_ftxt2)
# print(simple_ctxt2^simple_ftxt2)


c0 = simple_ctxt1[np.unravel_index(8, shape=(4,4), order='F')]
c13 = simple_ctxt1[np.unravel_index(5, shape=(4,4), order='F')]
c10 = simple_ctxt1[np.unravel_index(2, shape=(4,4), order='F')]
c7 = simple_ctxt1[np.unravel_index(15, shape=(4,4), order='F')]

c0_ = simple_ftxt1[np.unravel_index(8, shape=(4,4), order='F')]
c13_ = simple_ftxt1[np.unravel_index(5, shape=(4,4), order='F')]
c10_ = simple_ftxt1[np.unravel_index(2, shape=(4,4), order='F')]
c7_ = simple_ftxt1[np.unravel_index(15, shape=(4,4), order='F')]

c01 = simple_ctxt2[np.unravel_index(8, shape=(4,4), order='F')]
c131 = simple_ctxt2[np.unravel_index(5, shape=(4,4), order='F')]
c101 = simple_ctxt2[np.unravel_index(2, shape=(4,4), order='F')]
c71 = simple_ctxt2[np.unravel_index(15, shape=(4,4), order='F')]

c0_1 = simple_ftxt2[np.unravel_index(8, shape=(4,4), order='F')]
c13_1 = simple_ftxt2[np.unravel_index(5, shape=(4,4), order='F')]
c10_1 = simple_ftxt2[np.unravel_index(2, shape=(4,4), order='F')]
c7_1 = simple_ftxt2[np.unravel_index(15, shape=(4,4), order='F')]
print("third four bytes")
full_attack()

third four bytes


100%|██████████| 976/976 [01:03<00:00, 15.32it/s]


checking for correct keys


 69%|██████▉   | 664/960 [00:00<00:00, 3334.33it/s]

93 213 55 198


100%|██████████| 960/960 [00:00<00:00, 3417.48it/s]


In [28]:
#preperation for full attack

#for fourth column
simple_ctxt1 = all_ctext[0]
simple_ftxt1 = all_ftext[0]
simple_ctxt2 = all_ctext[8]
simple_ftxt2 = all_ftext[8]
c0 = simple_ctxt1[np.unravel_index(12, shape=(4,4), order='F')]
c13 = simple_ctxt1[np.unravel_index(9, shape=(4,4), order='F')]
c10 = simple_ctxt1[np.unravel_index(6, shape=(4,4), order='F')]
c7 = simple_ctxt1[np.unravel_index(3, shape=(4,4), order='F')]

c0_ = simple_ftxt1[np.unravel_index(12, shape=(4,4), order='F')]
c13_ = simple_ftxt1[np.unravel_index(9, shape=(4,4), order='F')]
c10_ = simple_ftxt1[np.unravel_index(6, shape=(4,4), order='F')]
c7_ = simple_ftxt1[np.unravel_index(3, shape=(4,4), order='F')]

c01 = simple_ctxt2[np.unravel_index(12, shape=(4,4), order='F')]
c131 = simple_ctxt2[np.unravel_index(9, shape=(4,4), order='F')]
c101 = simple_ctxt2[np.unravel_index(6, shape=(4,4), order='F')]
c71 = simple_ctxt2[np.unravel_index(3, shape=(4,4), order='F')]

c0_1 = simple_ftxt2[np.unravel_index(12, shape=(4,4), order='F')]
c13_1 = simple_ftxt2[np.unravel_index(9, shape=(4,4), order='F')]
c10_1 = simple_ftxt2[np.unravel_index(6, shape=(4,4), order='F')]
c7_1 = simple_ftxt2[np.unravel_index(3, shape=(4,4), order='F')]
print("fourth four bytes")
full_attack()

fourth four bytes


100%|██████████| 1288/1288 [01:29<00:00, 14.36it/s]


checking for correct keys


100%|██████████| 1280/1280 [00:00<00:00, 3656.63it/s]

170 35 50 172


In [249]:
# aeskeyschedule to find th eoreginal key

from aeskeyschedule import reverse_key_schedule,key_schedule

key_org= [168, 138, 164, 45, 53, 73, 46, 171 ,93, 213, 55, 198, 170, 35, 50, 172]
hex_1 = []
for i in range(16):
    hex_1.append(hex(key_org[i]))

print(hex_1)
# hex_1 = ['0xa8', '0x8a', '0xa4', '0x2d', '0x35', '0x49', '0x2e', '0xab', '0x5d', '0xd5', '0x37', '0xc6', '0xaa', '0x23', '0x32', '0xac']
# Assume key_org is the recovered key
key_org = [0xa8, 0x8a, 0xa4, 0x2d, 0x35, 0x49, 0x2e, 0xab, 0x5d, 0xd5, 0x37, 0xc6, 0xaa, 0x23, 0x32, 0xac]

# Generate the full key schedule from the recovered key
# key_s = key_schedule("0xa88aa42d35492eab5dd537c6aa2332ac")

# Rewind the key schedule to get the original key
key = reverse_key_schedule(key_org,10)
print(key)

# key_1 = reverse_key_schedule(b'^ul\xbd&U\xcc\xb0\xb3\xb5.\xf8F\xc5s\xb9',10)
# print(key_1)

['0xa8', '0x8a', '0xa4', '0x2d', '0x35', '0x49', '0x2e', '0xab', '0x5d', '0xd5', '0x37', '0xc6', '0xaa', '0x23', '0x32', '0xac']
b'^ul\xbd&U\xcc\xb0\xb3\xb5.\xf8F\xc5s\xb9'


In [252]:
!aeskeyschedule --round 10 a88aa42d35492eab5dd537c6aa2332ac

 0: 5e756cbd2655ccb0b3b52ef846c573b9
 1: f9fa3ae7dfaff6576c1ad8af2adfab16
 2: 65987d02ba378b55d62d53fafcf2f8ec
 3: e8d9b3b252ee38e784c36b1d783193f1
 4: 2705120e75eb2ae9f12841f48919d205
 5: e3b079a9965b5340677312b4ee6ac0b1
 6: c10ab1815751e2c13022f075de4830c4
 7: d30ead9c845f4f5db47dbf286a358fec
 8: c57d639e41222cc3f55f93eb9f6a1c07
 9: dce1a6459dc38a86689c196df7f6056a
10: a88aa42d35492eab5dd537c6aa2332ac


In [32]:
# After recovering the full key, decrypt the secret message:
# You can use the `pycryptodome` library 
from Crypto.Cipher import AES
from Crypto.Util.Padding import unpad
key_1 = bytes.fromhex('5e756cbd2655ccb0b3b52ef846c573b9')
print(key_1)
secret = bytes.fromhex("2a92fc6ad8006b658f49062c2843ad99")
print(secret)
cipher = AES.new(key_1, AES.MODE_ECB)  #  ciphering
original_msg = cipher.decrypt(secret)
print(original_msg)

b'^ul\xbd&U\xcc\xb0\xb3\xb5.\xf8F\xc5s\xb9'
b'*\x92\xfcj\xd8\x00ke\x8fI\x06,(C\xad\x99'
b'\xa7\xe1\xb1\nT\xce\xb6\xdb\xeb!\xcdtl\xa7yk'
